<img src="pandas_logo.png">

## Renombrar / Reordenar columnas
Muchas veces resulta interesante cambiar el nombre de una columna, después de realizar alguna operación sobre la misma o, simplemente por usabilidad.


In [6]:
import pandas as pd
import numpy as np

housing_df = pd.read_csv('datasets/housing.csv')

# El diccionario para renombrar presenta como claves las columnas originales y valores el nombre que le queremos dar
# como detalle usamos el parametro inplace, que nos permite cambiar el objeto sin tener que reinstanciarlo
housing_df.rename(columns = {'median_house_value' : 'median_house_price'}, inplace = True)

# Si no usasemos inplace = True y quisieramos guardar los cambios:
# housing_df = housing_df.rename(columns = {'median_house_value' : 'median_house_price'})

housing_df.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_price,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


Para mostrar como vambiar el orden de las columnas vamos a generar un nuevo dataframe solamente con longitud y latitud:

In [12]:
# Utilizamos el metodo copy() para generar un nuevo dataframe de la previa selección
housing_df_lat_lon = housing_df.loc[:, ['longitude', 'latitude']].copy()

In [18]:
housing_df_lat_lon.head(3)

,longitude,latitude
0,-122.23,37.88
1,-122.22,37.86
2,-122.24,37.85


In [17]:
# mostramos el df con las columnas cambiadas de orden
housing_df_lat_lon[['latitude', 'longitude']].head(3)  

,latitude,longitude
0,37.88,-122.23
1,37.86,-122.22
2,37.85,-122.24


## Borrar filas y columnas
Podemos borrar tanto filas como columnas. A continuación vemos un ejemplo borrando una fila mediante su indice y una columna mediante su nombre:

In [19]:
housing_df_lat_lon.drop(index = [0, 1]).head(3) # Vemos que la fila 0 y 1 no están

,longitude,latitude
2,-122.24,37.85
3,-122.25,37.85
4,-122.25,37.85


In [20]:
housing_df_lat_lon.drop(columns = ['longitude']).head(3) # Vemos que solo tenemos latitud

,latitude
0,37.88
1,37.86
2,37.85


## Gestión valores nulos

Vamos a ver cuales son las funciones que ofrece Pandas para manejar estos valores. 

Principalmente tenemos dos alternativas, borrar aquellas filas que tengan valores nulos (ahora veremos las posibilidades que tenemos) o rellenarlos de alguna forma.

In [23]:
housing_df.info()    # la columna total_bedrooms tiene valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_price    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


Para ver los ejemplos, vamos a generar valores nulos de manera aleatoria en ciertas filas de la columna **households**:

In [31]:
# generamos un numpy array con 300 valores aleatorios comprendidos entre 0 y 20639
indices_aleatorios = np.random.randint(low = 0, high = 20639, size = 300)

housing_df_mod = housing_df.copy()

# asignamos NA's a las filas seleccionadas en la columna households
housing_df_mod.loc[indices_aleatorios, 'households'] = np.nan

housing_df_mod.info()  # comprobamos que efectivamente se han actualizado esos valores

In [40]:
print(f'tamaño sin borrar: {housing_df_mod.shape}')

print()

# borramos las filas que contengan algun valor nulo
print(f"tamaño borrando con 'any': {housing_df_mod.dropna(how = 'any').shape}")
                                    
print()

# borramos las filas que tengan todos los valores del subset nulos (mucho mas restrictivo)
print(f"tamaño borrando con 'all': {housing_df_mod.dropna(how = 'all', subset = ['households', 'total_bedrooms']).shape}")

tamaño sin borrar: (20640, 10)

tamaño borrando con 'any': (20137, 10)

tamaño borrando con 'all': (20638, 10)


Supongamos que nos interesa en lugar de borrar esas filas, conservarlas. En este caso suelen seguirse alguna de las siguientes estrategias, aunque hay más:
    - Rellenar con un valor aleatorio
    - Rellenar con el valor más común de la columna (la moda): .mode()
    - Rellenar con la media o mediana: .mean() o .median()
        
Como ejemplo vamos a utilizar la media de la columna de total_bedrooms para quitar todos los valores nulos:

In [47]:
media_total_bedrooms = housing_df['total_bedrooms'].mean()

# Usamos la funcion fillna con la media y mostramos de nuevo la info para comprobar
housing_df.fillna(media_total_bedrooms).info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20640 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_price    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [42]:
housing_df_mod['total_bedrooms'].value_counts()


280.0     55
331.0     51
345.0     50
393.0     49
343.0     49
          ..
2205.0     1
1448.0     1
1691.0     1
2537.0     1
2546.0     1
Name: total_bedrooms, Length: 1923, dtype: int64